# Ezra's Project

In [1]:
import os     
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns 
import scipy as sc
from scipy.stats import chi2_contingency
import bs4
from bs4 import BeautifulSoup
import requests
from collections import Counter
import sklearn
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score
import re

In [2]:
from sklearn import preprocessing, linear_model, model_selection
from sklearn.model_selection import train_test_split

הפרוייקט ישאב נתוני מוצרים מאתרי האונליין של רשתות השיווק בארץ (שופרסל,וויקטורי).

המטרה הסופית - למידת מכונה שתאפשר לי לקבוע מחיר התחלתי למוצרים עבור סופר מתחרה. 

<b>Step 1:</b> Crawling

a. Shupersal online website. 

b. Victory online website. 

In [3]:
url= "https://www.shufersal.co.il/online/"
responsel= requests.get(url)

In [4]:
responsel

<Response [200]>

In [5]:
soup= BeautifulSoup(responsel.content, "html.parser")

 המוצרים באתר מסודרים על פי קטגוריות המופיעות בעמוד הראשון. בשלב ראשון של תהליך משיכת המידע, אמשוך טבלת קישורים הכוללת את שם הקטגוריה והקישור אל הדף עם המוצרים. 

In [6]:
tag=soup.find_all(attrs={"class":"forthMenu collapse"})
categories=[]
links=[]

In [7]:
for x in tag:
    href=x.find_all('a')
    for a in href:
        links.append(a['href'])
        categories.append(a.text.strip())
df=pd.DataFrame({"category":categories, "link":links})       
df

,category,link
0,ירקות,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...
1,פירות,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...
2,פיצוחים ופירות יבשים,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...
3,יוגורט ומשקאות יוגורט,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...
4,מעדנים וקינוחים,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...
...,...,...
231,מתוקים,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...
232,מוצרי נקיון אקולוגיים,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...
233,תוספי מזון,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...
234,ויטמינים,/online/he/%D7%A7%D7%98%D7%92%D7%95%D7%A8%D7%9...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  236 non-null    object
 1   link      236 non-null    object
dtypes: object(2)
memory usage: 3.8+ KB


In [9]:
# נדפיס את שמות כלל הקטגוריות שקיבלנו על מנת לראות שאכן כלל המידע הדרוש נמשך 
i=0
for x in df.category.unique():
    print (f"{i}) {x}") 
    i=i+1 

0) ירקות
1) פירות
2) פיצוחים ופירות יבשים
3) יוגורט ומשקאות יוגורט
4) מעדנים וקינוחים
5) מוצרי חלב וביצים
6) מדף הגבינות
7) גבינות מעדנייה
8) מוצרים לאפיה ובישול
9) תחליפי חלב וטופו
10) מוצרי עוף והודו
11) בשר בקר וכבש
12) דגים
13) מוצרי בשר "על האש"
14) מוצרים מוכנים לאכילה
15) מנגלים ומוצרים נילווים
16) לחמים, לחמניות ופיתות מהמאפייה
17) לחם, לחמניות ופיתות
18) מאפים ועוגות מהמאפיה
19) עוגות ארוזות
20) משקאות חריפים
21) בירה ומשקאות אנרגיה
22) יינות ותירוש
23) משקאות קלים
24) מים וסודה
25) תה וקפה
26) קפסולות ופולים לאספרסו
27) סיגריות וגפרורים
28) ירקות ופירות קפואים
29) מוצרי בשר, עוף ודגים קפואים
30) מוצרי בצק ומאפה קפוא
31) חומוס וסלטים
32) נקניקים ודגים מעושנים
33) קינוחים ומנות אחרונות
34) אוכל מוכן/ להכנה מהירה
35) תחליפי בשר קפוא
36) גלידות
37) קפואים לנמנעים מגלוטן
38) פסטה, אורז , קוסקוס וקטניות
39) רטבים ותוספות
40) שמן חומץ ומיץ לימון
41) שימורים
42) מוצרי יסוד ותבלינים
43) דבש ריבות וממרחים
44) מרקים, קרוטונים ותבשילים
45) מיוחדים
46) דגנים וחטיפי דגנים
47) ממתקים
48) חט

<b>פונקציית משיכת המידע ע"פ קטגוריה:</b>

In [10]:
## הפונקציה שמייצרת דאטה פריים של מוצרים עבור כל תת-קטגוריה לפי יחידה, משקל, חבילה ומוצר כפי שמסודר באתר  
def create_df1(category, url):
    ##
    links2=[]
    products=[] 
    prices=[] 
    sizes=[] 
    suplier=[] 
    responsel= requests.get(url)
    soup= BeautifulSoup(responsel.content, "html.parser")
    print(f"for {category} requst: {responsel}")
    tag1= soup.find_all("li", attrs={"class":"miglog-prod miglog-sellingmethod-by_unit"})
    tag2= soup.find_all("li", attrs={"class":"miglog-prod miglog-sellingmethod-by_weight"})
    tag3= soup.find_all("li", attrs={"class":"miglog-prod miglog-sellingmethod-by_package"})
    tag4= soup.find_all("li", attrs={"class":"miglog-prod"})
    tags=[tag1,tag2,tag3,tag4]
    for tag in tags:
        for li in tag: #product name:
            n1=li.find_all(attrs={"class":"textContainer"})
            n2=n1[0].find_all(attrs={"class":"text"})
            products.append(n2[0].text.strip())
            #product price: 
            
        for li in tag: # prices 
            n1=li.find_all(attrs={"class":"textContainer"})
            n2=n1[0].find_all(attrs={"class":"number"})
            prices.append(n2[0].text)
            
            # supplier and size 
        for li in tag:  
            n1=li.find_all(attrs={"class":"textContainer"})
            n3=n1[0].find_all(attrs={"class":"labelsListContainer"})
            n4=n3[0].find_all(attrs={"class":"smallText"})
            n5=n4[0].find_all("span") 
            if len(n5)>1:
                sizes.append(n5[0].text.strip())
                suplier.append(n5[1].text.strip())
            else: 
                sizes.append(None)
                suplier.append(None)

    
    df=pd.DataFrame({"super":"שופרסל","catgory":category,"product":products, "suplier":suplier, "sizes":sizes,"price":prices})
    return df 

<b>פונקצייה למשיכת כלל העמודים עבור כל קטגוריה:</b>

אציין שהאתר מסמלץ מעבר עמודים זמן גלילת העמוד מטה ואין צורך בלחיצה על פתור <b>עמ' הבא</b>"

הבנת החלוקה לעמודים בוצעה ע"י התבוננות בחלון הבדיקה בלשונית הרשת. 

In [11]:
def getDF_all_pages(category, url):
    responsel= requests.get(url)
    str_soup= str(BeautifulSoup(responsel.content, "html.parser"))
    num_of_pages=re.findall('data-pages="(\d\d*)"',str_soup)
    print(f"num of pages for {category} is : {num_of_pages[0]}")
    num=int(num_of_pages[0])
    extra="/fragment?q=:relevance&page="
    df_lst=[]
    for x in range(num):
        df=create_df1(category,f"{url}{extra}{x}")
        df_lst.append(df)
    df_lst= pd.concat(df_lst, axis=0)
    df_lst.reset_index(inplace=True)
    return df_lst

<b>דוגמא למשיכת מידע עבור קטגוריה אחת מטבלת הקישורים:</b> 

In [12]:
#ניסיון לבדיקה שהפונקציות אכן עובדות עבור לינק מהטבלת הקישורים 
try1= getDF_all_pages(df.iloc[0,0],f"{url}{df.iloc[0,1]}")

num of pages for ירקות is : 15
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>


In [13]:
try1 ## הטבלה שהתקבלה 

,index,super,catgory,product,suplier,sizes,price
0,0,שופרסל,ירקות,עגבניה שרי מידיפלם,שופרסל,460גרם,9.90
1,1,שופרסל,ירקות,עגבניה שרי תמר,שופרסל,460גרם,9.90
2,2,שופרסל,ירקות,עגבניה שרי אשכולות מובחר,שופרסל,460גרם,9.90
3,3,שופרסל,ירקות,עגבניה שרי יובל ליקופן,קטיף,460 גרם,9.90
4,4,שופרסל,ירקות,עגבניה שרי וויטני,שופרסל,460גרם,9.90
...,...,...,...,...,...,...,...
805,141,שופרסל,ירקות,עציץ בינוני,ליידי דיאן,יחידה,27.00
806,142,שופרסל,ירקות,עציץ קטן 2,ליידי דיאן,יחידה,17.00
807,143,שופרסל,ירקות,זר מיוחד מס' 15,ליידי דיאן,יחידה,75.00
808,144,שופרסל,ירקות,עציץ גדול,ליידי דיאן,יחידה,44.00


לאחר שבדקתי שהפונקציות עובדות אבצע אותן עבור כל אחת מהשורות בטבלת הקישורים על מנת לקבל טבלה אחודה. 
חריגים הם שורות 85,112,133 - עמודים אלה באתר בנויים באופן שונה משאר העמודים בטבלה, לאור העובדה שמדובר בשלוש קטגוריות בלבד החלטתי להתעלם מהן.

In [14]:
count=df.link.count()
temp_df=pd.DataFrame()
df_union=pd.DataFrame({"super":[],"catgory":[],"product":[], "suplier":[], "sizes":[],"price":[]})
for i in range(count):
    if((i!=85) and (i!=112)and (i!=133)):
        print(f"\n***********{df.iloc[i,0]}************\n**********{i}/{count}*********\n")
        temp_df=getDF_all_pages(df.iloc[i,0] ,f"{url}{df.iloc[i,1]}")
        df_union=pd.concat([df_union,temp_df], axis=0)


***********ירקות************
**********0/236*********

num of pages for ירקות is : 15
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>
for ירקות requst: <Response [200]>

***********פירות************
**********1/236*********

num of pages for פירות is : 4
for פירות requst: <Response [200]>
for פירות requst: <Response [200]>
for פירות requst: <Response [200]>
for פירות requst: <Response [200]>

***********פיצוחים ופירות יבשים************
**********2/236*********

num of pages for פיצוחים ופירות יבשים is : 12
for פיצוחים ופירות יבשים requst: <Response [2

num of pages for בירה ומשקאות אנרגיה is : 6
for בירה ומשקאות אנרגיה requst: <Response [200]>
for בירה ומשקאות אנרגיה requst: <Response [200]>
for בירה ומשקאות אנרגיה requst: <Response [200]>
for בירה ומשקאות אנרגיה requst: <Response [200]>
for בירה ומשקאות אנרגיה requst: <Response [200]>
for בירה ומשקאות אנרגיה requst: <Response [200]>

***********יינות ותירוש************
**********22/236*********

num of pages for יינות ותירוש is : 9
for יינות ותירוש requst: <Response [200]>
for יינות ותירוש requst: <Response [200]>
for יינות ותירוש requst: <Response [200]>
for יינות ותירוש requst: <Response [200]>
for יינות ותירוש requst: <Response [200]>
for יינות ותירוש requst: <Response [200]>
for יינות ותירוש requst: <Response [200]>
for יינות ותירוש requst: <Response [200]>
for יינות ותירוש requst: <Response [200]>

***********משקאות קלים************
**********23/236*********

num of pages for משקאות קלים is : 10
for משקאות קלים requst: <Response [200]>
for משקאות קלים requst: <Response [200]>
f

for שימורים requst: <Response [200]>
for שימורים requst: <Response [200]>
for שימורים requst: <Response [200]>
for שימורים requst: <Response [200]>
for שימורים requst: <Response [200]>

***********מוצרי יסוד ותבלינים************
**********42/236*********

num of pages for מוצרי יסוד ותבלינים is : 15
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסוד ותבלינים requst: <Response [200]>
for מוצרי יסו

for לתינוק- מזון, חיתולים ומגבונים requst: <Response [200]>
for לתינוק- מזון, חיתולים ומגבונים requst: <Response [200]>
for לתינוק- מזון, חיתולים ומגבונים requst: <Response [200]>
for לתינוק- מזון, חיתולים ומגבונים requst: <Response [200]>

***********לתינוק- טיפוח, הלבשה ואביזרים************
**********57/236*********

num of pages for לתינוק- טיפוח, הלבשה ואביזרים is : 18
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים requst: <Response [200]>
for לתינוק- טיפוח, הלבשה ואביזרים 

for מוצרים אקולוגיים requst: <Response [200]>
for מוצרים אקולוגיים requst: <Response [200]>

***********מוצרים לבישול ואפיה************
**********73/236*********

num of pages for מוצרים לבישול ואפיה is : 17
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצרים לבישול ואפיה requst: <Response [200]>
for מוצר

...

for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>
for צעצועים ומשחקים requst: <Response [200]>

***********ציוד לגן ולבית ספר************
**********145/236*********

num of pages for ציוד לגן ולבית ספר is : 9
for ציוד לגן ולבית ספר requst: <Response [200]>
for ציוד לגן ולבית ספר requst: <Response [200]>
for ציוד לגן ולבית ספר requst: <Response [200]>
for ציוד לגן ולבית ספר requst: <Response [200]>
for ציוד לגן ולבית ספר requst: <Response [200]>
for ציוד לגן ולב

for בישום נשים requst: <Response [200]>
for בישום נשים requst: <Response [200]>
for בישום נשים requst: <Response [200]>
for בישום נשים requst: <Response [200]>

***********בישום גברים************
**********160/236*********

num of pages for בישום גברים is : 17
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>
for בישום גברים requst: <Response [200]>

***********בשמי בוטיק************
*******

for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>
for טיפוח טבעי requst: <Response [200]>

***********טיפוח ועיצוב שיער************
**********172/236*********

num of pages for טיפוח ועיצוב שיער is : 13
for טיפוח ועיצוב שיער requst: <Response [200]>
for טיפוח ועיצוב שיער requst: <Response [200]>
for טיפוח ועיצוב שיער requst: <Response [200]>
for טיפוח ועיצוב שיער requst: <Response [200]>
for טיפוח ועיצוב שיער requst: <Response [200]>
for טיפוח ועיצוב שיער requst: <Response [200]>
for טיפוח ועיצוב שיער requst: <Response [200]

for סבון,שמפו ומרכך requst: <Response [200]>
for סבון,שמפו ומרכך requst: <Response [200]>
for סבון,שמפו ומרכך requst: <Response [200]>
for סבון,שמפו ומרכך requst: <Response [200]>
for סבון,שמפו ומרכך requst: <Response [200]>
for סבון,שמפו ומרכך requst: <Response [200]>

***********היגיינה************
**********188/236*********

num of pages for היגיינה is : 5
for היגיינה requst: <Response [200]>
for היגיינה requst: <Response [200]>
for היגיינה requst: <Response [200]>
for היגיינה requst: <Response [200]>
for היגיינה requst: <Response [200]>

***********דאודורנט************
**********189/236*********

num of pages for דאודורנט is : 4
for דאודורנט requst: <Response [200]>
for דאודורנט requst: <Response [200]>
for דאודורנט requst: <Response [200]>
for דאודורנט requst: <Response [200]>

***********היגיינת פה************
**********190/236*********

num of pages for היגיינת פה is : 16
for היגיינת פה requst: <Response [200]>
for היגיינת פה requst: <Response [200]>
for היגיינת פה requst: <Resp

for אובלטים ומצות requst: <Response [200]>

***********חלב, גבינות ויוגורט************
**********216/236*********

num of pages for חלב, גבינות ויוגורט is : 2
for חלב, גבינות ויוגורט requst: <Response [200]>
for חלב, גבינות ויוגורט requst: <Response [200]>

***********משקאות, גבינות וטופו תחליפיים************
**********217/236*********

num of pages for משקאות, גבינות וטופו תחליפיים is : 5
for משקאות, גבינות וטופו תחליפיים requst: <Response [200]>
for משקאות, גבינות וטופו תחליפיים requst: <Response [200]>
for משקאות, גבינות וטופו תחליפיים requst: <Response [200]>
for משקאות, גבינות וטופו תחליפיים requst: <Response [200]>
for משקאות, גבינות וטופו תחליפיים requst: <Response [200]>

***********ביצים אורגניות************
**********218/236*********

num of pages for ביצים אורגניות is : 1
for ביצים אורגניות requst: <Response [200]>

***********ממרחים ופשתן************
**********219/236*********

num of pages for ממרחים ופשתן is : 1
for ממרחים ופשתן requst: <Response [200]>

***********ללא גל

# וקיבלנו את הטבלה עבור שופרסל 

<b>
נבצע טיפול ראשוני בנתונים: 
1. הסרת העמודה אינדקס 
2. המרת עמודת המחיר למספרי 
3. המשך הטיפול ייעשה לאחר שאאחד את הטבלאות מכלל המקורות 
    </b>

In [15]:
mydf=df_union.copy()
mydf.reset_index(inplace=True)
mydf.drop(['index','level_0'],axis=1,inplace=True)
mydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74431 entries, 0 to 74430
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   super    74431 non-null  object
 1   catgory  74431 non-null  object
 2   product  74431 non-null  object
 3   suplier  69799 non-null  object
 4   sizes    69799 non-null  object
 5   price    74431 non-null  object
dtypes: object(6)
memory usage: 3.4+ MB


In [16]:
for p in range(mydf.price.count()):
    mydf.price[p]=mydf.price[p].replace(",","")
    mydf.price[p]=mydf.price[p].replace("החל מ-","")
mydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74431 entries, 0 to 74430
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   super    74431 non-null  object
 1   catgory  74431 non-null  object
 2   product  74431 non-null  object
 3   suplier  69799 non-null  object
 4   sizes    69799 non-null  object
 5   price    74431 non-null  object
dtypes: object(6)
memory usage: 3.4+ MB


In [17]:
mydf.price= mydf.price.astype('float')

In [18]:
mydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74431 entries, 0 to 74430
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   super    74431 non-null  object 
 1   catgory  74431 non-null  object 
 2   product  74431 non-null  object 
 3   suplier  69799 non-null  object 
 4   sizes    69799 non-null  object 
 5   price    74431 non-null  float64
dtypes: float64(1), object(5)
memory usage: 3.4+ MB


<b>בשלב זה אחלץ את הטבלה לאקסל על מנת שלא אצטרך בכל פעם לשאוב את הנתונים. 

In [19]:
mydf.to_excel('shupersal_df_u.xlsx',encoding='utf-8')

In [20]:
shupersal_df=pd.read_excel("shupersal_df_u.xlsx")

In [21]:
shupersal_df.head()

,Unnamed: 0,super,catgory,product,suplier,sizes,price
0,0,שופרסל,ירקות,עגבניה שרי מידיפלם,שופרסל,460גרם,9.9
1,1,שופרסל,ירקות,עגבניה שרי תמר,שופרסל,460גרם,9.9
2,2,שופרסל,ירקות,עגבניה שרי אשכולות מובחר,שופרסל,460גרם,9.9
3,3,שופרסל,ירקות,עגבניה שרי יובל ליקופן,קטיף,460 גרם,9.9
4,4,שופרסל,ירקות,עגבניה שרי וויטני,שופרסל,460גרם,9.9


# כעת נבצע שלבים דומים עבור מקור שונה - אתר הקניות של וויקטורי אונליין 

In [22]:
url= "https://www.victoryonline.co.il/%D7%9C%D7%97%D7%9D/%D7%9E%D7%93%D7%A3"
responsel= requests.get(url)

In [23]:
responsel

<Response [200]>

In [24]:
soup= BeautifulSoup(responsel.content, "html.parser")

In [25]:
tag=soup.find_all(attrs={"class":"DepartmentLinks"})
categories=[]
links=[]

In [26]:
for x in tag:
    href=x.find_all('a')
    for a in href:
        links.append(a['href'])
        categories.append(a.text.strip())
df=pd.DataFrame({"category":categories, "link":links})       
df

,category,link
0,המועדפים שלי,https://www.victoryonline.co.il/המועדפים_שלי/מדף
1,קניות קודמות,https://www.victoryonline.co.il/YourAccount/Or...
2,רשימות,https://www.victoryonline.co.il/רשימות/מדף
3,מבצעים מומלצים,https://www.victoryonline.co.il/מבצעים_מומלצים...
4,כל המבצעים,https://www.victoryonline.co.il/כל_המבצעים/מדף
...,...,...
194,אביזרי אמבט,https://www.victoryonline.co.il/אביזרי_אמבט/מדף
195,סבון מוצק,https://www.victoryonline.co.il/סבון_מוצק/מדף
196,סבון נוזלי,https://www.victoryonline.co.il/סבון_נוזלי/מדף
197,מוצרי ספיגה למבוגרים,https://www.victoryonline.co.il/מוצרי_ספיגה_למ...


קיבלנו טבלת קישורים. 
נסתכל על הקטגוריות, נשווה לאתר ונסדר את הטבלה בהתאם לצורך. 

In [27]:
i=0
for c in df.category:
    print(f"{i}) {c}")
    i=i+1

0) המועדפים שלי
1) קניות קודמות
2) רשימות
3) מבצעים מומלצים
4) כל המבצעים
5) מבצעי כרטיס אשראי ויקטורי
6) מבצעי 5 שקלים
7) חדש על המדף
8) חנות מותג סנו
9) חנות מותג טבעול
10) חנות מוצרי חלב
11) מתחם טיפוח
12) חנות יקבי כרמל
13) 
14) ללא גלוטן
15) טבעוני
16) אורגני
17) לחם
18) לחמניות ופיתות
19) פריכיות, מצות ותחליפי לחם
20) מאפייה טריה
21) פירות וירקות
22) פירות
23) ירקות
24) פירות יבשים, אגוזים ופיצוחים
25) חלב, משקאות ותחליפי חלב
26) חלב ומשקאות
27) משקאות סויה ואורז
28) גבינות
29) קוטג' וגבינה לבנה
30) בולגרית, פטה, ומלוחה
31) גבינה צהובה ומוצרלה
32) גבינות מחלב עיזים או כבשים
33) גבינות קשות
34) גבינות רכות
35) גבינת שמנת
36) יוגורט ומעדנים
37) יוגורט
38) מעדנים
39) בשר, עוף והודו
40) בשר טרי
41) עוף והודו טרי
42) בשר ועוף מוכן
43) דגים טריים וארוזים
44) דגים טריים
45) דגים ארוזים וסושי טרי
46) נקניקים ונקניקיות
47) נקניקיות
48) נקניקים ופסטרמה
49) בישול ואפיה
50) חמאה, מרגרינה
51) שמנת ולבן
52) שמנת לבישול והקצפה
53) סלטים
54) פסטה טריה
55) ביצים
56) טופו וסייטן
57) חומוס וטחינה
5

In [28]:
df=df.iloc[17:,:]

In [29]:
df.reset_index(inplace=True)
df.drop(['index'],axis=1,inplace=True)
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,category,link
0,לחם,https://www.victoryonline.co.il/לחם/מדף
1,לחמניות ופיתות,https://www.victoryonline.co.il/לחמניות_ופיתות...
2,"פריכיות, מצות ותחליפי לחם",https://www.victoryonline.co.il/פריכיות_מצות_ו...
3,מאפייה טריה,https://www.victoryonline.co.il/מאפייה_טריה/מדף
4,פירות וירקות,https://www.victoryonline.co.il/פירות_וירקות/מדף
...,...,...
177,אביזרי אמבט,https://www.victoryonline.co.il/אביזרי_אמבט/מדף
178,סבון מוצק,https://www.victoryonline.co.il/סבון_מוצק/מדף
179,סבון נוזלי,https://www.victoryonline.co.il/סבון_נוזלי/מדף
180,מוצרי ספיגה למבוגרים,https://www.victoryonline.co.il/מוצרי_ספיגה_למ...


<b> נשים לב שמוצר עלול להופיע בכפילות - בקטגוריה ראשית ובקטגוריה משנית, נתייחס לכך בסידור הנתונים 

In [30]:
#  עבור וויקטורי אונליין הפונקציה שמייצרת דאטה פריים של מוצרים  
def create_df2(category, url):
    ##
    products=[] 
    prices=[] 
    sizes=[] 
    suplier1=[] 
    responsel= requests.get(url)
    soup1= BeautifulSoup(responsel.content, "html.parser")
    print(f"for {category} requst: {responsel}")
    tag1= soup1.find_all("li", attrs={"class":"NgMspProductCell MSM Expanded"})
    
    for t in tag1:
        names=t.find_all(attrs={"class":"Prefix"})
        products.append(names[0].text.strip())

    for t in tag1:
        size=t.find_all(attrs={"class":"Suffix"})
        sizes.append(size[0].text.strip())
 
    for t in tag1:
        Price=t.find_all(attrs={"class":"Price"})
        prices.append(Price[0].text.strip())
    
    df=pd.DataFrame({"super":"וויקטורי","catgory":category,"product":products, "suplier":None, "sizes":sizes,"price":prices})
    
    tag2 = soup1.find_all(attrs={"class":"FilterGroup","groupid":"brand"})
    suplier_lst=[]
    
    pattern = "חנות מותג (\w*\s*\w*)"
    pattern2= "חנות (\w*\s*\w*)"
    if(re.findall("חנות מותג",category)):
        df["suplier"]=re.findall(pattern,category)[0]
        return df
    if(re.findall("חנות",category)):
        df["suplier"]=re.findall(pattern2,category)[0]
        return df
    
    
    if len(tag2)>0:
        sup=tag2[0].find_all("a")
        for s in sup:
            suplier_lst.append(s.text.strip())
        print(suplier_lst)
        for x in range(df["product"].count()):
            for s in suplier_lst:
                if (re.findall((f" {s}"),df["product"][x])):
                    df["suplier"][x]=s
                
    return df 

<b> נשים לב - עמודת הספק נשאבת מעמודת שם המוצר, זאת לאור העובדה שנתון זה אינו מופרד באתר  ממנו נלקחים הפרטים 
    
    נבדוק שהפונקציה אכן עובדת
    
    

In [31]:
create_df2(df.iloc[6,0],df.iloc[6,1])

for ירקות requst: <Response [200]>
['<מותג פרטי>', 'א.אדמה', 'ויקטורי', 'חסלט', 'מרינה', 'שטראוס']


,super,catgory,product,suplier,sizes,price
0,וויקטורי,ירקות,בצל אדום יבש לפי משקל,None,"(500 גרם, כ- 3-4 יח')",₪6.90
1,וויקטורי,ירקות,בצל יבש לפי משקל,None,"(500 גרם, כ- 3-5 יח')",₪4.90
2,וויקטורי,ירקות,בצל ירוק א.אדמה,א.אדמה,,₪5.90
3,וויקטורי,ירקות,בצל ירוק אוצר הארץ מהדרין חסלט,חסלט,,₪7.90
4,וויקטורי,ירקות,בצל ירוק כפרי טרי חסלט,חסלט,,₪7.90
...,...,...,...,...,...,...
91,וויקטורי,ירקות,טעם הטבע שעועית אדומה מבושלת שטראוס,שטראוס,(475 גרם),₪14.90
92,וויקטורי,ירקות,בטטה לפי משקל,None,"(500 גרם, כ - 1 יח')",₪8.90
93,וויקטורי,ירקות,תפוח אדמה אדום תפזורת לפי משקל,None,"(500 גרם, כ- 2-4 יח')",₪4.90
94,וויקטורי,ירקות,תפוח אדמה לבן ארוז לפי משקל,None,(500 גרם),₪3.90


נכתוב פונקציה למשיכת כלל העמודים. כאן הדבר היה פשוט יותר לאור העובדה שעמוד מכיל עד 100 מוצרים ומופיע סמן העמ' הבא. 

In [32]:
def getDF_all_pages2(category, url):
    responsel= requests.get(url)
    soup=BeautifulSoup(responsel.content, "html.parser")
    
    of_product=soup.find_all(attrs={"class":"ListTitleWrap"})
    num_of_product=of_product[0].find_all(attrs={"id":"ProductCount","class":"ProductCount"})[0].text.strip()
    num_of_product=num_of_product.replace("(","")
    num_of_product=num_of_product.replace(")","")
    print(f"num of product for {category} is : {num_of_product}")
    num=int(num_of_product)//100+1
    print(f"num of pages for {category} is : {num}")
    extra="?Page="
    df_lst=[]
    for x in range(1,num+1):
        df=create_df2(category,f"{url}{extra}{x}")
        df_lst.append(df)
    df_lst= pd.concat(df_lst, axis=0)
    df_lst.reset_index(inplace=True)
    return df_lst

<b>וכעת נשאב את כלל הנתונים עבור המוצרים באתר וויקטורי </b>

In [33]:
count=df.link.count()
temp_df=pd.DataFrame()
df_union=pd.DataFrame({"super":[],"catgory":[],"product":[], "suplier":[], "sizes":[],"price":[]})
for i in range(count):
    print(f"\n***********{df.iloc[i,0]}************\n**********{i}/{count}*********\n")
    temp_df=getDF_all_pages2(df.iloc[i,0] ,df.iloc[i,1])
    df_union=pd.concat([df_union,temp_df], axis=0)


***********לחם************
**********0/182*********

num of product for לחם is : 48
num of pages for לחם is : 1
for לחם requst: <Response [200]>
['אחדות', "אנג'ל", "בונז'ור", 'ברמן', 'גרין לייט', 'דגנית עין בר', 'לחם הארץ']

***********לחמניות ופיתות************
**********1/182*********

num of product for לחמניות ופיתות is : 24
num of pages for לחמניות ופיתות is : 1
for לחמניות ופיתות requst: <Response [200]>
["אנג'ל", 'ברמן', 'גרין לייט', 'דגנית עין בר', 'לחם הארץ', 'מאפיית אריאל', 'מאפיית ודש', 'מעדני תנעמי', 'פיתה אקספרס']

***********פריכיות, מצות ותחליפי לחם************
**********2/182*********

num of product for פריכיות, מצות ותחליפי לחם is : 54
num of pages for פריכיות, מצות ותחליפי לחם is : 1
for פריכיות, מצות ותחליפי לחם requst: <Response [200]>
['אבאל', "אנרג'י", 'אסם', 'אקסטרה (תבליני מימון)', 'וילי פוד', 'זן בר', 'מאסטר שף', 'מנה', 'נסטלה', 'פתית', 'ריל פודס']

***********מאפייה טריה************
**********3/182*********

num of product for מאפייה טריה is : 6
num of pages

num of product for יוגורט is : 170
num of pages for יוגורט is : 2
for יוגורט requst: <Response [200]>
['גד', 'גמדים', 'דנונה', 'המחלבה', 'חלב הארץ', 'יופלה', 'יורו', 'מולר', 'משק צוריאל', 'פרילי', 'תנובה']
for יוגורט requst: <Response [200]>
['גד', 'גמדים', 'דנונה', 'המחלבה', 'חלב הארץ', 'יופלה', 'יורו', 'מולר', 'משק צוריאל', 'פרילי', 'תנובה']

***********מעדנים************
**********21/182*********

num of product for מעדנים is : 72
num of pages for מעדנים is : 1
for מעדנים requst: <Response [200]>
['אלפרו', 'באדי', 'גד', 'גמדים', 'יולו', 'מוו', 'מולר', 'מילקי', 'ספלנדיד', 'סקי', 'עדן קינוחים', 'קרלו', 'שטראוס', 'תנובה', 'תנובה אלטרנטיב']

***********בשר, עוף והודו************
**********22/182*********

num of product for בשר, עוף והודו is : 90
num of pages for בשר, עוף והודו is : 1
for בשר, עוף והודו requst: <Response [200]>
['<מותג פרטי>', 'אדום אדום', 'מאמא עוף', 'נחמיה לחוביץ', 'נטו', 'עטרה', 'שאטובריאן']

***********בשר טרי************
**********23/182*********

num of product fo

num of product for שניצל בשרי וארוחות מוכנות בשריות is : 34
num of pages for שניצל בשרי וארוחות מוכנות בשריות is : 1
for שניצל בשרי וארוחות מוכנות בשריות requst: <Response [200]>
['טיבון ויל', 'מאמא עוף', 'עוף טוב']

***********גלידות וקינוחים************
**********52/182*********

num of product for גלידות וקינוחים is : 3
num of pages for גלידות וקינוחים is : 1
for גלידות וקינוחים requst: <Response [200]>
["בראנץ' אנד קראנץ'", 'כרמית', 'פינוקים (קפוא)']

***********חנות מותג טבעול************
**********53/182*********

num of product for חנות מותג טבעול is : 35
num of pages for חנות מותג טבעול is : 1
for חנות מותג טבעול requst: <Response [200]>

***********מרקים ותבשילים************
**********54/182*********

num of product for מרקים ותבשילים is : 31
num of pages for מרקים ותבשילים is : 1
for מרקים ותבשילים requst: <Response [200]>
['אסם', 'דודלס', 'וילי פוד', "מג'יק(נטו)", 'מנה', 'פולדס', 'קנור']

***********תבשילים************
**********55/182*********

num of product for תבשילים is

for ריבות requst: <Response [200]>
['778', 'יד מרדכי', 'מנז אנד גאסר', 'סאן דלפור', 'ענבי ציון']

***********ממרחים מתוקים************
**********79/182*********

num of product for ממרחים מתוקים is : 31
num of pages for ממרחים מתוקים is : 1
for ממרחים מתוקים requst: <Response [200]>
['אחוה', 'דולכור', 'הנמל', 'השחר העולה', 'השקד', 'לוטוס', 'מובנפיק', 'מיה', 'נוטלה', 'סקיפי', 'עלית', 'קליית גת', 'שקדיה', 'תמר כנרת']

***********ממרחים אחרים************
**********80/182*********

num of product for ממרחים אחרים is : 12
num of pages for ממרחים אחרים is : 1
for ממרחים אחרים requst: <Response [200]>
['אוליביה', 'טבסקו', 'טוסו', 'טעמי אסיה', 'פרסקו', 'תבליני ביטון יוחאי']

***********אפיה ביתית************
**********81/182*********

num of product for אפיה ביתית is : 118
num of pages for אפיה ביתית is : 2
for אפיה ביתית requst: <Response [200]>
['אופנהיימר', 'אסם', 'אשבל', 'בונומי', 'בריאות וטבע דגש', 'הנמל', 'השחר העולה', 'ויליגר', 'טחנת קמח דגן', 'טחנת רוט', 'יד מרדכי', 'מיה', 'מימונס', 'מ

num of product for עוגות is : 36
num of pages for עוגות is : 1
for עוגות requst: <Response [200]>
['אחוה', 'אסם', "דולצ'ה ויטה", 'יוגטה', 'מילקי', 'סבן דייז', 'עלית', 'פייבר וואן', "צ'וקטה", 'קינדר', 'קפולסקי', "ריץ'"]

***********עוגיות************
**********99/182*********

num of product for עוגיות is : 76
num of pages for עוגיות is : 1
for עוגיות requst: <Response [200]>
['אוראו', 'אחוה', 'אסם', 'באלזן', 'בונומי', "דולצ'ה ויטה", 'טוסו', 'כרמית', 'לה בייקרי', 'לחמי', 'מילקה', 'מן', 'מעדני מלך', 'מרבה', 'נסטלה', 'סבן דייז', 'עבאדי', 'עלית', 'קונדיטוריית אנגל', 'קפולסקי', 'שיבולים']

***********סוכריות ומסטיקים************
**********100/182*********

num of product for סוכריות ומסטיקים is : 44
num of pages for סוכריות ומסטיקים is : 1
for סוכריות ומסטיקים requst: <Response [200]>
['אורביט', 'איירוייבס', 'בזוקה', 'הובה בובה', 'יוגטה', 'מאסט', 'מנטוס', 'סקיטלס', 'עלית', 'פז', 'פייב', 'שטורק']

***********שוקולד************
**********101/182*********

num of product for שוקולד is : 120
nu

for משקאות קרים requst: <Response [200]>
['אלוורה קינג', 'אקווה נובה', 'אקסל', "בורג'מי", 'בלו', "ג'ומקס", 'היל ויו', "ויטמינצ'יק", 'טמפו', 'יכין', 'כרמית', 'מונסטר', 'מי עדן', 'מירינדה', 'נביעות', 'נטורה נובה', 'נשר מאלט', 'סאן בנדטו', 'סאן פלגרינו', 'סאפה', 'סבן אפ', 'סוויטנגו', 'ספרייט', 'ספרינג', 'עין גדי', 'עלית', 'פאנטה', 'פיוז-טי', 'פפסי', "פרוט אנד וג'", 'פרוט ווטר', 'פריגת', 'פרימור(מצונן)', 'פריניב', 'קוקה-קולה', 'קינלי סודה', 'קריסטל', 'רד בול', 'שוופס', 'תפוזינה']
for משקאות קרים requst: <Response [200]>
['אלוורה קינג', 'אקווה נובה', 'אקסל', "בורג'מי", 'בלו', "ג'ומקס", 'היל ויו', "ויטמינצ'יק", 'טמפו', 'יכין', 'כרמית', 'מונסטר', 'מי עדן', 'מירינדה', 'נביעות', 'נטורה נובה', 'נשר מאלט', 'סאן בנדטו', 'סאן פלגרינו', 'סאפה', 'סבן אפ', 'סוויטנגו', 'ספרייט', 'ספרינג', 'עין גדי', 'עלית', 'פאנטה', 'פיוז-טי', 'פפסי', "פרוט אנד וג'", 'פרוט ווטר', 'פריגת', 'פרימור(מצונן)', 'פריניב', 'קוקה-קולה', 'קינלי סודה', 'קריסטל', 'רד בול', 'שוופס', 'תפוזינה']

***********אבקות שוקו ושתיה**********

num of product for חומרי ניקיון לבית is : 58
num of pages for חומרי ניקיון לבית is : 1
for חומרי ניקיון לבית requst: <Response [200]>
["אג'קס", 'אסטוניש', 'דזיטול', 'יעקבי', 'כלורוקס', 'סיף', 'סנו', 'סנט מוריץ', 'שניב']

***********אביזרי ניקיון************
**********144/182*********

num of product for אביזרי ניקיון is : 70
num of pages for אביזרי ניקיון is : 1
for אביזרי ניקיון requst: <Response [200]>
['אלבגינו', 'אסותא', 'ביאנקו', 'דיש מטיק', 'וי טריל', 'יוגב', 'יעקבי', 'מבריקל הפלא', 'ניקול', 'סנו', 'סנו סושי', 'סנט מוריץ', 'עיסא ערטול', 'קוויק', 'ריצפז', 'שייני']

***********ניקוי כלים************
**********145/182*********

num of product for ניקוי כלים is : 48
num of pages for ניקוי כלים is : 1
for ניקוי כלים requst: <Response [200]>
['סוד', 'סנו', 'פיירי', 'פיניש', 'פלמוליב', 'שניב']

***********ניקוי רצפות************
**********146/182*********

num of product for ניקוי רצפות is : 27
num of pages for ניקוי רצפות is : 1
for ניקוי רצפות requst: <Response [200]>
['נקה', 'סוד', '

num of product for טיפוח ועיצוב השיער is : 2
num of pages for טיפוח ועיצוב השיער is : 1
for טיפוח ועיצוב השיער requst: <Response [200]>
['נטורל פורמולה', 'פנטן']

***********מרכך שיער ומסיכות************
**********175/182*********

num of product for מרכך שיער ומסיכות is : 39
num of pages for מרכך שיער ומסיכות is : 1
for מרכך שיער ומסיכות requst: <Response [200]>
['הוואי', 'כיף', 'לוריאל', 'נטורל פורמולה', 'נקה 7', 'סטייל', 'פינוק', 'פנטן', 'קרליין']

***********סבונים והיגיינת הגוף************
**********176/182*********

num of product for סבונים והיגיינת הגוף is : 100
num of pages for סבונים והיגיינת הגוף is : 2
for סבונים והיגיינת הגוף requst: <Response [200]>
['דאב', 'דר. פישר', 'הוואי', 'כיף', 'נובו', 'נקה 7', 'פינוק', 'פלמוליב', 'פמילי', 'פנינה רוזנבלום', 'קרמה']
for סבונים והיגיינת הגוף requst: <Response [200]>
['דאב', 'דר. פישר', 'הוואי', 'כיף', 'נובו', 'נקה 7', 'פינוק', 'פלמוליב', 'פמילי', 'פנינה רוזנבלום', 'קרמה']

***********אביזרי אמבט************
**********177/182*********

In [34]:
mydf2=df_union.copy()

In [35]:
mydf2

,super,catgory,product,suplier,sizes,price,index
0,וויקטורי,לחם,לחם אחיד פרוס אנג'ל,אנג'ל,(750 גרם),₪6.60,0.0
1,וויקטורי,לחם,לחם חיטה מלאה אנג'ל,אנג'ל,(750 גרם),₪14.90,1.0
2,וויקטורי,לחם,לחם מחמצת בסגנון איטלקי אנג'ל,אנג'ל,(500 גרם),₪15.90,2.0
3,וויקטורי,לחם,לחם עינן קל פרוס אנג'ל,אנג'ל,(500 גרם),₪14.80,3.0
4,וויקטורי,לחם,לחם פרוס 100% קמח מלא קל בתוספת שיפון אנג'ל,אנג'ל,(750 גרם),₪12.90,4.0
...,...,...,...,...,...,...,...
84,וויקטורי,סבון נוזלי,אל סבון פרחי בר ורוד פמילי,פמילי,(1 ליטר),₪14.90,84.0
85,וויקטורי,סבון נוזלי,סבון נוזלי לידיים מועשר בשמן זית פמילי,פמילי,"(500 מ""ל)",₪9.90,85.0
86,וויקטורי,סבון נוזלי,אלכוג'ל לעור הידיים דר. פישר,דר. פישר,"(500 מ""ל)",₪29.90,86.0
87,וויקטורי,סבון נוזלי,אלכוג'ל לעור הידיים עם כיסוי דמות מסיליקון דר....,דר. פישר,"(30 מ""ל)",₪19.90,87.0


In [36]:
mydf2.drop(["index"],axis=1, inplace=True)

In [37]:
mydf2

,super,catgory,product,suplier,sizes,price
0,וויקטורי,לחם,לחם אחיד פרוס אנג'ל,אנג'ל,(750 גרם),₪6.60
1,וויקטורי,לחם,לחם חיטה מלאה אנג'ל,אנג'ל,(750 גרם),₪14.90
2,וויקטורי,לחם,לחם מחמצת בסגנון איטלקי אנג'ל,אנג'ל,(500 גרם),₪15.90
3,וויקטורי,לחם,לחם עינן קל פרוס אנג'ל,אנג'ל,(500 גרם),₪14.80
4,וויקטורי,לחם,לחם פרוס 100% קמח מלא קל בתוספת שיפון אנג'ל,אנג'ל,(750 גרם),₪12.90
...,...,...,...,...,...,...
84,וויקטורי,סבון נוזלי,אל סבון פרחי בר ורוד פמילי,פמילי,(1 ליטר),₪14.90
85,וויקטורי,סבון נוזלי,סבון נוזלי לידיים מועשר בשמן זית פמילי,פמילי,"(500 מ""ל)",₪9.90
86,וויקטורי,סבון נוזלי,אלכוג'ל לעור הידיים דר. פישר,דר. פישר,"(500 מ""ל)",₪29.90
87,וויקטורי,סבון נוזלי,אלכוג'ל לעור הידיים עם כיסוי דמות מסיליקון דר....,דר. פישר,"(30 מ""ל)",₪19.90


# וקיבלנו את הטבלה עבור וויקטורי. 

<b> נייצא אותה לאקסל ונטען על מנת לחסוך את שלב שאיבת המידע כל פעם</b>

In [38]:
mydf2.to_excel('victory.xlsx',encoding='utf-8')